In [66]:
import pandas as pd
import numpy as np
import os
import glob
import nltk.data
import nltk, re, pprint
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.metrics.pairwise import linear_kernel
import sqlite3



def connect_db():
	return sqlite3.connect('/Users/sheldon/podcasts/test.db')

def create_df_object():
	conn = sqlite3.connect('/Users/sheldon/podcasts/test.db')
	df = pd.read_sql("select * from podcast",conn)
	return df

df = create_df_object()

stop = set(stopwords.words('english'))
'''
def remove_stop_words(row):
    tokens = word_tokenize(str(row))
    tokens = [w for w in tokens if not w in stop]
    tokens = [word for word in tokens if not "'" in word]
    return ' '.join(tokens)

df['transcribed'] = df['transcribed'].apply(remove_stop_words)

def get_top_words(row):
    tokens = word_tokenize(str(row))
    counter = nltk.FreqDist(tokens)
    return counter.most_common(25)

df['top_words'] =  df.transcribed.apply(get_top_words)
#tf = TfidfVectorizer()
'''
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3),min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(df['transcribed'])
copy_matrix = tf.transform(df['transcribed'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [67]:
indices = np.argsort(tf.idf_)[::-1]
feature_names = tf.get_feature_names()
top_n = 25
top_features = [feature_names[i] for i in indices[:top_n]]
print top_features

'''lectures = ["this is some food", "this is some drink"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(lectures)
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]
print top_features'''

[u'zwerdling mouth trade', u'good news slip', u'good nice', u'good newsof radiation', u'good newsof', u'good news world', u'good news went', u'good news week', u'good news ve', u'good news twin', u'good news thinks', u'good news temper', u'good news storybook', u'good news sees', u'good news hiring', u'good news second', u'good news san', u'good news right', u'good news recommendations', u'good news opening', u'good news okay', u'good news new', u'good news media', u'good news little', u'good news light']


'lectures = ["this is some food", "this is some drink"]\nvectorizer = TfidfVectorizer()\nX = vectorizer.fit_transform(lectures)\nindices = np.argsort(vectorizer.idf_)[::-1]\nfeatures = vectorizer.get_feature_names()\ntop_n = 2\ntop_features = [features[i] for i in indices[:top_n]]\nprint top_features'

In [ ]:
#trying genism

In [68]:
mat_array = tfidf_matrix.toarray()
for l in mat_array: 
    print [(feature_names[x],l[x]) for x in (l*-1).argsort()][:10][0]

(u'like', 0.24071265870416453)
(u'like', 0.16953929221919495)
(u'like', 0.15178888577344432)
(u'like', 0.26599877507194869)
(u'like', 0.18111983626961034)
(u'like', 0.20079463693112398)
(u'like', 0.18904763206697089)
(u'like', 0.1687774777269041)
(u'like', 0.2273983932752662)
(u'tommy', 0.27021064224583458)
(u'chandler', 0.22907846454480191)
(u'elkins', 0.21303314852461949)
(u'julius', 0.093043818165899753)
(u'petrified', 0.25628525064879321)
(u'gender', 0.19378055701049737)
(u'danny', 0.073621992122139171)
(u'venus fly', 0.12386446234490764)
(u'trophy', 0.098003605694790982)
(u'crime', 0.12148540114184396)
(u'saint louis', 0.1033595476667932)
(u'economists', 0.1598993269489119)
(u'gift', 0.21192991044398757)
(u'sister linda', 0.088409940193915737)
(u'financial', 0.21247624919260338)
(u'people', 0.078036207111245232)
(u'obesity', 0.24769765180761563)
(u'coin', 0.32437614040657131)
(u'offender', 0.26261421441127869)
(u'kidney', 0.29931945484214911)
(u'preference', 0.10314730490472233)
(

KeyboardInterrupt: 

In [46]:
podcast_id = 3
conn = sqlite3.connect('/Users/sheldon/podcasts/test.db')
c = conn.cursor()
cur = c.execute('select "index", episode, series from podcast where "index" ={}'.format(podcast_id))
db_request = [dict(index=row[0], episode=row[1], series=row[2]) for row in cur.fetchall()]

related_pod_index = cosine_similarities.argsort()[podcast_id][::-1]
pod_dict = dict(zip(range(0, len(related_pod_index)),related_pod_index))
pod_dict = pd.DataFrame({'rank':pod_dict.keys()}, index=pod_dict.values())
related_podcasts_df = pd.DataFrame.join(pod_dict, df, how='inner')
final_df = related_podcasts_df.sort_values('rank')[1:11][['rank','episode','series','top_words']]
related_podcasts = final_df['episode']

In [50]:
with pd.option_context('display.max_rows', 999, 'display.max_columns', 3):
    print final_df.top_words

0      [(like, 124), (know, 53), (think, 39), (get, 3...
5      [(like, 103), (know, 58), (get, 42), (one, 38)...
116    [(like, 198), (know, 90), (one, 63), (get, 58)...
150    [(like, 234), (would, 165), (know, 164), (good...
7      [(like, 88), (get, 54), (one, 44), (know, 37),...
1      [(like, 85), (know, 49), (get, 39), (man, 37),...
113    [(like, 157), (people, 94), (know, 80), (reall...
6      [(like, 83), (know, 35), (yeah, 28), (one, 26)...
4      [(like, 73), (know, 34), (get, 29), (one, 22),...
8      [(like, 108), (one, 33), (know, 31), (get, 28)...
Name: top_words, dtype: object


In [28]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
'''
stopset = set(stopwords.words('english'))

with open('sentiment_test.txt', 'r') as text_file:
    text = text_file.read()
    tokens=word_tokenize(str(text))
    tokens = [w for w in tokens if not w in stopset]
    print tokens
    
'''

def remove_stop_get_count(row):
    tokens = word_tokenize(str(row))
    tokens = [w for w in tokens if not w in stop]
    tokens = [word for word in tokens if not "'" in word]
    counter = nltk.FreqDist(tokens)
    return counter.most_common(25)

test['tokenized'] = test.transcribed.apply(remove_stop_get_count)

/Users/sheldon/anaconda/envs/capstone/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [48]:
def remove_stop_words(row):
    tokens = word_tokenize(str(row))
    tokens = [w for w in tokens if not w in stop]
    tokens = [word for word in tokens if not "'" in word]
    return ' '.join(tokens)

x = df['transcribed'].ix[0]
df['transcribed'].ix[0]


'queens supine milton ten minute rama world news every e-mail newsletter embarking mason milch e-mails easy segmentation improving campaign season eighty testing measure performance advanced analytics melted snow better e. mail know queens supported swiss based helping anyone bring ideas life line beautiful way learn free trial swiss based com slash queen whoa nice east indians many dad got back bite going take kids instead sees inexact algebra homework think want know super jack old ugly treadmill thing think obviously freaky ca read boring yeah lay working gets boring slightly yeah every goodness think found back fit end abeyance carousel academy bury us empty bill please deputy c. studios ca believe finally may knew coolly food fifth came idea fudo queens awhile ago man seabees taking something day shall enough instilling much chemistry cameras trigger bombing dad hand upon budzyn bernardino name regis inhalants trying show together really funny began much fines news lying tenderly 

In [22]:
fdist.most_common(25)

[("'s", 126),
 ('like', 124),
 ("n't", 85),
 ('know', 53),
 ('think', 39),
 ('get', 37),
 ('one', 36),
 ("'m", 30),
 ("'re", 28),
 ('white', 24),
 ('man', 23),
 ('really', 23),
 ("'d", 22),
 ('yeah', 21),
 ('see', 20),
 ('said', 17),
 ('night', 17),
 ('right', 17),
 ('love', 17),
 ('want', 16),
 ('mean', 16),
 ('back', 16),
 ('ice', 15),
 ('oh', 15),
 ('cream', 15)]